
## Sliderule 3DEP sampling
### From Scott's Arctic DEM sampling

In [26]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import boto3
from sliderule import sliderule, icesat2, earthdata

In [27]:
#https://xyzservices.readthedocs.io/en/stable/introduction.html
import xyzservices



In [28]:
aoi_url = 'https://raw.githubusercontent.com/ICESAT-2HackWeek/dzdt/main/sites/grandmesa.geojson'
gfa = gpd.read_file(aoi_url)
terrain_map = xyzservices.providers.Stamen.Terrain()
gfa.explore(style_kwds=dict(fill=False, color='magenta'),tiles=terrain_map)

In [29]:
icesat2.init("slideruleearth.io", verbose=True)

In [30]:
# Start with all defaults, just Polygon
parms = {
    "poly": sliderule.toregion(gfa)["poly"],
    # "srt": icesat2.SRT_LAND,
    # "cnf": icesat2.CNF_SURFACE_HIGH,
    # "len": 40.0,
    # "res": 20.0,
    # "maxi": 6
    "t0": "2023-01-01T00:00:00Z",
    "t1": "2024-01-01T00:00:00Z",
}

In [31]:
# Add Raster Sampling
# https://www.slideruleearth.io/web/rtd/user_guide/GeoRaster.html
geojson = earthdata.tnm(short_name='Digital Elevation Model (DEM) 1 meter', polygon=sliderule.toregion(gfa)['poly']) 
parms["samples"] = {
            # "hls": {"asset": "landsat-hls", "bands": ["NDSI"]},
            "dem_stats": {"asset": "usgs3dep-1meter-dem", "radius":10.0, "zonal_stats":True, "catalog":geojson},
            #"mosaic": {"asset": "arcticdem-mosaic", "algorithm":"NearestNeighbour"},
            #"mosaic": {"asset": "arcticdem-mosaic", "radius": 10.0, "zonal_stats": True},
            #"strips": {"asset": "arcticdem-strips", "algorithm": "CubicSpline"}
        }

# From JP Swinski
geojson = earthdata.tnm(short_name='Digital Elevation Model (DEM) 1 meter', polygon=region) 
parms = {
    "poly": region,
    "samples" : {
            "dem_stats": {"asset": "usgs3dep-1meter-dem", "catalog": geojson}
    }
}


### Run sliderule

In [32]:
parms

{'poly': [{'lon': -108.15996262442047, 'lat': 39.00669498695481},
  {'lon': -108.12553849524684, 'lat': 39.00828408047391},
  {'lon': -108.12826515894373, 'lat': 39.03979372594071},
  {'lon': -108.21688172909325, 'lat': 39.0847831386408},
  {'lon': -108.23562754200954, 'lat': 39.06176272688245},
  {'lon': -108.2424442012517, 'lat': 39.05249952359773},
  {'lon': -108.19506841951798, 'lat': 39.01199182651902},
  {'lon': -108.15996262442047, 'lat': 39.00669498695481}],
 't0': '2023-01-01T00:00:00Z',
 't1': '2024-01-01T00:00:00Z',
 'samples': {'dem_stats': {'asset': 'usgs3dep-1meter-dem',
   'radius': 10.0,
   'zonal_stats': True,
   'catalog': '{"type": "FeatureCollection", "features": [{"type": "Feature", "id": "5eaa520782cefae35a222095", "geometry": {"type": "Polygon", "coordinates": [[[-108.340806218, 38.9960647394], [-108.228673333, 38.9960647394], [-108.228673333, 39.088876958], [-108.340806218, 39.088876958], [-108.340806218, 38.9960647394]]]}, "properties": {"datetime": "2020-11-17

In [33]:
%%time

# Run SlideRule

gf = icesat2.atl06p(parms, version='006')

print("Start:", gf.index.min().strftime('%Y-%m-%d'))
print("End:", gf.index.max().strftime('%Y-%m-%d'))
print("Reference Ground Tracks: {}".format(gf["rgt"].unique()))
print("Cycles: {}".format(gf["cycle"].unique()))
print("Elevation Measurements: {} ".format(gf.shape[0]))
gf.head(2)

INFO:sliderule.earthdata:Identified 2 resources to process
INFO:sliderule.sliderule:request <AppServer.4281> processing initialized on ATL03_20230206192127_07371802_006_01.h5 ...
INFO:sliderule.sliderule:request <AppServer.4263> processing initialized on ATL03_20230306061322_11561806_006_01.h5 ...
INFO:sliderule.sliderule:request <AppServer.4263> processing of ATL03_20230306061322_11561806_006_01.h5 complete (0/0/0)
INFO:sliderule.sliderule:request <AppServer.4263> processing complete (0/0/0/0)
INFO:sliderule.sliderule:request <AppServer.4281> processing of ATL03_20230206192127_07371802_006_01.h5 complete (228548/38/0)
INFO:sliderule.sliderule:Successfully completed processing resource [2 out of 2]: ATL03_20230306061322_11561806_006_01.h5
INFO:sliderule.sliderule:request <AppServer.4281> processing complete (197/110/284/0)
INFO:sliderule.sliderule:Successfully completed processing resource [1 out of 2]: ATL03_20230206192127_07371802_006_01.h5


Start: 2023-02-06
End: 2023-02-06
Reference Ground Tracks: [737]
Cycles: [18]
Elevation Measurements: 284 
CPU times: user 121 ms, sys: 3.35 ms, total: 124 ms
Wall time: 11.2 s


,rgt,dh_fit_dx,h_sigma,dh_fit_dy,w_surface_window_final,pflags,spot,rms_misfit,cycle,h_mean,...,dem_stats.mad,dem_stats.value,dem_stats.file_id,dem_stats.flags,dem_stats.median,dem_stats.count,dem_stats.max,dem_stats.stdev,dem_stats.time,dem_stats.min
time,,,,,,,,,,,,,,,,,,,,,
2023-02-06 19:24:34.552389888,737,-0.221149,0.359040,0.0,5.672367,0,6,1.136813,18,3098.086954,...,0.407346,3074.377703,0,0,3074.420916,317,3074.987567,0.481194,1.289672e+09,3073.202899
2023-02-06 19:24:34.555207936,737,0.644488,0.531968,0.0,30.000000,0,6,4.249098,18,3100.922838,...,0.150399,3074.926981,0,0,3074.875711,317,3075.377909,0.207850,1.289672e+09,3074.095926


In [35]:
gf[['dem_stats.median']].describe()

,dem_stats.median
count,284.000000
mean,3081.501818
std,34.416126
min,2966.053970
25%,3076.666644
50%,3092.903680
75%,3103.657947
max,3114.276160


In [12]:
gf.keys()

Index(['rgt', 'dh_fit_dx', 'h_sigma', 'dh_fit_dy', 'w_surface_window_final',
       'pflags', 'spot', 'rms_misfit', 'cycle', 'h_mean', 'gt', 'distance',
       'n_fit_photons', 'segment_id', 'geometry'],
      dtype='object')

## Sliderule ArcticDEM sampling

In [14]:
aoi_url = 'https://raw.githubusercontent.com/ICESAT-2HackWeek/dzdt/main/sites/Anaktuvuk_north_slope_smaller.geojson'
gfa = gpd.read_file(aoi_url)
terrain_map = xyzservices.providers.Stamen.Terrain()
gfa.explore(style_kwds=dict(fill=False, color='magenta'),tiles=terrain_map)

In [15]:
icesat2.init("slideruleearth.io", verbose=True)

In [16]:
# Start with all defaults, just Polygon
parms = {
    "poly": sliderule.toregion(gfa)["poly"],
    # "srt": icesat2.SRT_LAND,
    # "cnf": icesat2.CNF_SURFACE_HIGH,
    # "len": 40.0,
    # "res": 20.0,
    # "maxi": 6
    "t0": "2023-01-01T00:00:00Z",
    "t1": "2024-01-01T00:00:00Z",
}

In [18]:
# Add Raster Sampling
# https://www.slideruleearth.io/web/rtd/user_guide/GeoRaster.html
#geojson = earthdata.tnm(short_name='Digital Elevation Model (DEM) 1 meter', polygon=sliderule.toregion(gfa)['poly']) 
parms["samples"] = {
            # "hls": {"asset": "landsat-hls", "bands": ["NDSI"]},
            #"dem_stats": {"asset": "usgs3dep-1meter-dem", "radius":10.0, "zonal_stats":True, "catlog":geojson},
            "mosaic": {"asset": "arcticdem-mosaic", "algorithm":"NearestNeighbour"},
            #"mosaic": {"asset": "arcticdem-mosaic", "radius": 10.0, "zonal_stats": True},
            #"strips": {"asset": "arcticdem-strips", "algorithm": "CubicSpline"}
        }

In [19]:
parms

{'poly': [{'lon': -151.87367192270008, 'lat': 68.43894683572523},
  {'lon': -151.23536889834176, 'lat': 68.43894683572523},
  {'lon': -151.23536889834176, 'lat': 68.79422334370639},
  {'lon': -151.87367192270008, 'lat': 68.79422334370639},
  {'lon': -151.87367192270008, 'lat': 68.43894683572523}],
 't0': '2023-01-01T00:00:00Z',
 't1': '2024-01-01T00:00:00Z',
 'samples': {'mosaic': {'asset': 'arcticdem-mosaic',
   'algorithm': 'NearestNeighbour'}}}

In [20]:
%%time

# Run SlideRule

gf = icesat2.atl06p(parms, version='006')

print("Start:", gf.index.min().strftime('%Y-%m-%d'))
print("End:", gf.index.max().strftime('%Y-%m-%d'))
print("Reference Ground Tracks: {}".format(gf["rgt"].unique()))
print("Cycles: {}".format(gf["cycle"].unique()))
print("Elevation Measurements: {} ".format(gf.shape[0]))
gf.head(2)

INFO:sliderule.earthdata:Identified 10 resources to process
INFO:sliderule.sliderule:request <AppServer.4223> processing initialized on ATL03_20230111115559_03351805_006_01.h5 ...
INFO:sliderule.sliderule:request <AppServer.4155> processing initialized on ATL03_20230220212212_09521803_006_01.h5 ...
INFO:sliderule.sliderule:request <AppServer.4166> processing initialized on ATL03_20230416072735_03961905_006_02.h5 ...
INFO:sliderule.sliderule:request <AppServer.4409> processing initialized on ATL03_20230412073542_03351905_006_01.h5 ...
INFO:sliderule.sliderule:request <AppServer.4114> processing initialized on ATL03_20230314085944_12801805_006_01.h5 ...
INFO:sliderule.sliderule:request <AppServer.4208> processing initialized on ATL03_20230115114743_03961805_006_01.h5 ...
INFO:sliderule.sliderule:request <AppServer.4168> processing initialized on ATL03_20230213102350_08381805_006_01.h5 ...
INFO:sliderule.sliderule:request <AppServer.4123> processing initialized on ATL03_20230321195821_000

Start: 2023-01-11
End: 2023-04-16
Reference Ground Tracks: [ 335  396  449  510  838  952 1280    7]
Cycles: [18 19]
Elevation Measurements: 50839 
CPU times: user 3.22 s, sys: 127 ms, total: 3.35 s
Wall time: 32.3 s


,rgt,dh_fit_dx,h_sigma,dh_fit_dy,w_surface_window_final,pflags,spot,rms_misfit,cycle,h_mean,gt,distance,n_fit_photons,segment_id,geometry,mosaic.flags,mosaic.time,mosaic.file_id,mosaic.value
time,,,,,,,,,,,,,,,,,,,
2023-01-11 11:58:58.868576768,335,0.066361,0.053765,0.0,3.0,0,2,0.394529,18,450.589235,50,1.240978e+07,54,619545,POINT (-151.23542 68.69930),0,1.358109e+12,0,451.066132
2023-01-11 11:58:58.871392256,335,0.062722,0.059284,0.0,3.0,0,2,0.439641,18,451.810011,50,1.240980e+07,55,619546,POINT (-151.23547 68.69912),0,1.358109e+12,0,452.202332


In [25]:
gf[['mosaic.value']].describe()

,mosaic.value
count,50839.000000
mean,608.754802
std,117.881949
min,264.641602
25%,525.040466
50%,604.303589
75%,687.247284
max,969.982178
